In [7]:
# get all act traj and sample sizes
import os
from collections import defaultdict
from shutil import copyfile, rmtree
import json

def get_path_lens(adir):
    tp = defaultdict(list)

    for x in os.listdir(adir):
        traj_path = os.path.join(adir, x)
        print(traj_path)
        for i in ids_we_care_about:
            inst_path = os.path.join(traj_path, str(i))
            if os.path.isdir(inst_path):
                for y in os.listdir(inst_path):
                    ip = os.path.join(inst_path, y)
                    num_imgs = len(os.listdir(os.path.join(ip, 'rgb')))
    #                 print(f'{ip}, len {num_imgs}')
                    tp[i].append((ip, num_imgs))
    return tp

# print(tp)

def acopyjson(a, b, sample, ctr):
    with open(a, "r") as f:
        dat = json.load(f)
        
    dat2 = {}
    
    if os.path.isfile(b):
        with open(b, "r") as f:
            dat2 = json.load(f)
    
    og_len = len(dat.keys())
    
    x = 0
    sampled = 0
    fstep = og_len/sample
    
    while True:
        dat2.update({ctr: dat[str(x)]})
        x = round(x + fstep)
        ctr += 1
        sampled += 1
        x = round(sampled*fstep)
        if x >= og_len or sampled >= sample:
            break
        
    with open(b, "w") as fp:
        json.dump(dat2, fp)


def acopydir(a, b, ext, sample, ctr):
#     print(f'acopydir {r}')
    if not os.path.isdir(b):
        os.makedirs(b) 
        
    # convert len(a) things to sample things    
    og_len = len(os.listdir(a))
    x = 0 
    sampled = 0
    fstep = og_len/sample
    
    while True:
        f = "{:05d}{}".format(x, ext)
#         print(f)
        copyfile(os.path.join(a,f), os.path.join(b,"{:05d}{}".format(ctr, ext)))
#         print(f'x {x}, ctr {ctr}, fstep {fstep}')
        ctr += 1
        sampled += 1
        x = round(sampled*fstep)

        if x >= og_len or sampled >= sample:
            break
            
    print(f'sampled {sampled}, expected {sample}')
            
    

def move(out_dir, src_dir, l):
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        os.makedirs(os.path.join(out_dir, 'rgb'))
        os.makedirs(os.path.join(out_dir, 'seg'))
        os.makedirs(os.path.join(out_dir, 'depth'))
        
    print(f'{out_dir, src_dir, l}')
    
    ctr = len(os.listdir(os.path.join(out_dir, 'rgb')))
    
    acopydir(os.path.join(src_dir, 'rgb'), os.path.join(out_dir, 'rgb'), '.jpg', l, ctr)
    acopydir(os.path.join(src_dir, 'depth'), os.path.join(out_dir, 'depth'), '.npy', l, ctr)
    acopydir(os.path.join(src_dir, 'seg'), os.path.join(out_dir, 'seg'), '.npy', l, ctr)
    acopyjson(os.path.join(src_dir, 'data.json'), os.path.join(out_dir, 'data.json'), l, ctr)
    
    assert len(os.listdir(os.path.join(out_dir, 'rgb'))) - ctr == l    

def subsample_and_move(atrajs, btrajs, out_dir):
    assert len(atrajs) == len(btrajs)
    
    for x in range(len(atrajs)):
        pa, la = atrajs[x]
        pb, lb = btrajs[x]
        
        minl = min(la, lb)
        
        print(f'moving {minl} things for {pa}, {lb} for {pb}')
        
        move(os.path.join(out_dir, 'active'), pa, minl)
        move(os.path.join(out_dir, 'baseline'), pb, lb)


import random
random.seed(7)

act_dir = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_traj_man_1116_train'
def_dir = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116'

def_dir_test = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116_cvprtest'
act_dir_test = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_traj_man_1116_test'

sample_sizes = [1]


ids_we_care_about = [243,404,196,133,166,170,172] #[404,196,133]

act_lens = get_path_lens(act_dir)
def_lens = get_path_lens(def_dir)

for k, v in act_lens.items():
    print(k, v)

/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_traj_man_1116_train/4
/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_traj_man_1116_train/8
/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_traj_man_1116_train/2
/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_traj_man_1116_train/3
/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_traj_man_1116_train/5
/checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116/1
/checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116/2
/checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116/3
/checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116/4
243 [('/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_traj_man_1116_train/4/243/1', 61)]
404 [('/checkpoi

In [8]:
# for s in sample_sizes:
s = 3
out_dir = f'/checkpoint/apratik/data_devfair0187/apartment_0/instance_det_sampled{s}_1116_cvpr2_train3'

if os.path.isdir(out_dir):
    print(f'rmtree {out_dir}')
    rmtree(out_dir)

print(f'out_dir for sample size {s}: {out_dir}')

# prep s trajectories from each, subsample
for i in ids_we_care_about:
    act_sample = min(s, len(list(act_lens[i])))
    def_sample = min(s, len(list(def_lens[i])))

    ss = min(act_sample, def_sample)
    print(f'\nsampling {ss} trajectories for each')

    act_trajs = random.sample(list(act_lens[i]), ss)
    def_trajs = random.sample(list(def_lens[i]), ss)

    subsample_and_move(act_trajs, def_trajs, out_dir)
        
    
    

out_dir for sample size 3: /checkpoint/apratik/data_devfair0187/apartment_0/instance_det_sampled3_1116_cvpr2_train3

sampling 1 trajectories for each
moving 61 things for /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_traj_man_1116_train/4/243/1, 75 for /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116/2/243/1
('/checkpoint/apratik/data_devfair0187/apartment_0/instance_det_sampled3_1116_cvpr2_train3/active', '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_traj_man_1116_train/4/243/1', 61)
sampled 61, expected 61
sampled 61, expected 61
sampled 61, expected 61
('/checkpoint/apratik/data_devfair0187/apartment_0/instance_det_sampled3_1116_cvpr2_train3/baseline', '/checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_trajectories_1116/2/243/1', 75)
sampled 75, expected 75
sampled 75, expected 75
sampled 75, expected 75

sampling 3 trajectories for each
moving 10 things for /c